# CHAPTER 7. Optimizing and Tuning Spark Applications
> 6장에서는 스파크가 어떻게 메모리 관리를 하고, 고급 API 를 통해서 데이터셋을 구성하는 지에 대해 학습했으며, 이번 장에서는 최적화를 위한 스파크 설정과, 조인 전략들을 살펴보고, 스파크 UI 를 통해 안좋은 영향을 줄 수 있는 것들에 대한 힌트를 얻고자 합니다.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)


## 7.1 Optimizing and Tuning Spark for Efficiency
> 스파크는 [튜닝](https://spark.apache.org/docs/latest/tuning.html)을 위한 다양한 설정을 제공하며, [설정](https://spark.apache.org/docs/latest/configuration.html)값을 통해 확인할 수 있습니다

### 7.1.1 Viewing and Setting Apache Spark Configurations
> 아래의 순서대로 스파크는 설정값을 읽어들이며, 가장 마지막에 변경된 값이 반영됩니다

#### 1. 설치된 스파크 경로의 conf/spark-default.conf 파일을 생성 및 수정

#### 2. 스파크 실행 시에 옵션을 지정하는 방법
```bash
$ spark-submit --conf spark.sql.shuffle.partitions=5 --conf "spark.executor.memory=2g" --class main.scala.chapter7.SparkConfig_7_1 jars/mainscala-chapter7_2.12-1.0.jar
```

#### 3. 스파크 코드 내에서 직접 지정하는 방법
```scala
SparkSession.builder
.config("spark.sql.shuffle.partitions", 5)
.config("spark.executor.memroy", "2g")
...
```

In [18]:
# 파이스파크 내에서는 sparkContext 를 통해서 해당 정보를 가져올 수 있습니다
def printConfigs(session):
    for x in sorted(session.sparkContext.getConf().getAll()):
        print(x)

printConfigs(spark)

('spark.app.id', 'local-1619328594851')
('spark.app.name', 'pyspark-shell')
('spark.driver.host', 'ddd9c0cd39a3')
('spark.driver.port', '42405')
('spark.executor.id', 'driver')
('spark.master', 'local[*]')
('spark.rdd.compress', 'True')
('spark.serializer.objectStreamReset', '100')
('spark.sql.session.timeZone', 'Asia/Seoul')
('spark.submit.deployMode', 'client')
('spark.submit.pyFiles', '')
('spark.ui.showConsoleProgress', 'true')


In [71]:
# SparkSQL의 경우 내부적으로 사용되는 설정값이 다르기 때문에 더 많은 정보가 출력됩니다
spark.sql("SET -v").select("key", "value").where("key like '%spark.sql%'").show(n=5, truncate=False)

+---------------------------------------------------------+----------------------------------------------------------------+
|key                                                      |value                                                           |
+---------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes          |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.coalescePartitions.enabled            |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum|<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionNum    |<undefined>                                                     |
|spark.sql.adaptive.enabled                               |false                                                           |


* 스파크 UI 를 통해서도 확인이 가능합니다

![spakr-ui](images/spark-ui.png)

In [70]:
# 스파크 기본 설정 spark.sql.shuffle.partitions 값을 확인하고, 프로그램 상에서 변경 후 테스트 합니다
num_partitions = spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.shuffle.partitions", 5)
mod_partitions = spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.shuffle.partitions", num_partitions)
print(num_partitions, mod_partitions)

200 5


### 7.1.2 Scaling Spark for Large Workloads

#### 1. 정적 vs 동적 리소스 할당의 선택
> CPU 및 Memory 사용을 애플리케이션에 따라 지정하는 정적 리소스 할당과 동적 리소스 할당은 처리해야 할 데이터의 특성에 따라 선택할 수 있으며, 환경설정을 다르게 구성해야 합니다.

* 데이터의 크기가 일정하지 않고, 유동적
* 특히 데이터의 크기가 고르지 않은 스트리밍 처리
* 멀티테넌시 환경의 분석용 클러스터의 데이터 리소스 관리

#### 2. 동적 리소스 할당 설정 가이드
* 기본 설정은 false 이므로 아래의 값들에 대한 설정이 별도로 되어야 하며, REPL 환경에서 지원하지 않는 값들도 존재하므로, 프로그램을 통한 수정이 필요합니다
```
spark.dynamicAllocation.enabled true
spark.dynamicAllocation.minExecutors 2
spark.dynamicAllocation.schedulerBacklogTimeout 1m
spark.dynamicAllocation.maxExecutors 20
spark.dynamicAllocation.executorIdleTimeout 2min
```
* 아래의 과정을 통해 동적 리소스를 관리합니다
  - 1. 스파크 드라이버가 클러스터 매니저에 2개(minExecutors)의 익스큐터를 요청합니다
  - 2. 작업 큐의 백로그가 증가하여, 백로그 타임아웃(schedulerBacklogTimeout)이 발생하는 경우 새로운 익스큐터 요청이 발생합니다
  - 3. 스케줄링 된 작업들이 1분 이상 지연되는 경우 드라이버는 새로운 익스큐터를 최대 20개(maxExecutors) 까지 요청합니다
  - 4. 스파크 드라이버는 2분 이상 (executorIdleTimeout) 작업이 할당되지 않는 익스큐터 들을 종료시킵니다

#### 3. 스파크 익스큐터의 메모리와 셔플 서비스의 설정 가이드
![external-memory-layout](images/external-memory-layout.png)
* 맵, 스필 그리고 병합 프로세스들이 I/O 부족에 따른 문제점을 갖지 않으며, 최종 셔플 파티션이 디스크에 저장되기 전에 버퍼 메모리를 확보할 수 있도록 설정을 아래와 같이 조정할 수 있습니다
![spark-conf-io](images/spark-conf-io.png)

#### 4. 스파크 병렬성을 최대화
> 스파크가 데이터를 어떻게 저장소로부터 메모리에 적재하는지, 스파크에 있어서 파티션이 어떻게 활용되는지를 이해해야 합니다

* 매 스테이지 마다 많은 타스크들이 존재하지만, 스파크는 기껏해야 코어당 작업당 하나의 스레드만 할당하며, 개별 타스크는 독립된 파티션 하나를 처리합니다.
* 리소스 사용을 최적화하고, 병렬성을 최대화 하려면 익스큐터에 존재하는 코어수들 만큼 많은 파티션들이 존재해야 합니다. (유휴 코어를 두지 않기 위함)
![figure.7-3](images/figure.7-3.png)


#### 파티션은 어떻게 구성되는가?
* 분산 저장소에 저장시에 구성되는 경우
  - HDFS, S3 등의 저장소의 기본 파일블록의 크기는 64mb, 128mb 이며, 파일 크기가 작고 많아질 수록 파티션당 할당해야 하는 코어수가 모자라기 때문에 "small file problem" 을 피해야 합니다
* 스파크의 셔플링을 통해 생성되는 경우
  - 집계함수나 조인과 같은 Wide Transformation 과정에서 셔플링이 발생 (Network & Disk I/O 비용)
  - 기본 셔플 파티션 수는 200개인데 작은 데이터집합이나, 스트리밍 워크로드 등에는 **충분히 많은 수이기 때문에 조정이 필요**합니다
* 


#### 질문과 답변
* 대부분 dynamic allocation 을 쓰면 좋을거 아닐까?
  - 워크로드가 예상된다면 동적할당은 필요없는 리소스 및 관리 비용이 더 들어가기 때문에 성능에 영향을 줄 수 있습니다
* REPL 이 뭔가?
  - Read-Evaluate-Print Loop 의 약자
* dynamic allocation 은 수시로 변경할 수 없는가? 왜 그런가?
* off-heap 이 좋으면 모두 off-heap 사용하지 왜 jvm 메모리를 이렇게나 많이 사용하는가?
* execution vs storage 메모리의 비율을 어떻게 확인할 수 있는가? 오히려 삽질 아닌가?
* spark 작업에서의 spill 절차는 무엇이고 왜 발생하며 어떻게 해결할 수 있는가?
  - 스파크 익스큐터가 위의 각 레이어에 할당된 메모리를 모두 사용한 경우 디스크로 저장하는 경우를 Spill 이라고 합니다
  - Disk I/O 는 성능에 큰 영향을 미치기 때문에 SSD 를 사용한다면 좋은 성능을 효과를 기대할 수 있습니다
```
operations, the shuffle will spill results to executors’ local disks at the location specified in spark.local.directory. Having performant SSD disks for this operation will boost the performance.
```

In [5]:
numDF = spark.range(1000).repartition(16)
numDF.rdd.getNumPartitions()

16


## 7.2 Caching and Persistence of Data
### 7.2.1 DataFrame.cache()



### 7.2.2 DataFrame.persist()


### 7.2.3 When to Cache and Persist


### 7.2.4 When Not to Cache and Persist



## 7.3 A Family of Spark Joins
### 7.3.1 Broadcast Hash Join



### 7.3.2 Shuffle Sort Merge Join



## 7.4 Inspecting the Spark UI
### 7.4.1 Journey Through the Spark UI Tabs


## 7.5 Summary


## 추가로 학습할 내용들
* [Tuning Apache Spark for Large Scale Workloads - Sital Kedia & Gaoxiang Liu](https://www.youtube.com/watch?v=5dga0UT4RI8)
* [Hive Bucketing in Apache Spark - Tejas Patil](https://www.youtube.com/watch?v=6BD-Vv-ViBw)
* [How does Facebook tune Apache Spark for Large-Scale Workloads?](https://towardsdatascience.com/how-does-facebook-tune-apache-spark-for-large-scale-workloads-3238ddda0830)
* [Spark Internal Part 2. Spark의 메모리 관리(2)](https://medium.com/@leeyh0216/spark-internal-part-2-spark%EC%9D%98-%EB%A9%94%EB%AA%A8%EB%A6%AC-%EA%B4%80%EB%A6%AC-2-db1975b74d2f)
* [Why You Should Care about Data Layout in the Filesystem](https://databricks.com/session/why-you-should-care-about-data-layout-in-the-filesystem)